  # Metrics

# Configuration

In [1]:
import os

from definitions import poem_1, poem_2
from utils import settings, gemini, ollama3, gpt4, opus, init_gemini
import utils

%load_ext jupyter_ai_magics

In [2]:
#settings
temperature = 0.8
system_prompt = "You are an expert in German literature. You answer the questions truthfully and short."

settings(system_prompt, temperature)
    

In [3]:
#defining aliases
init_gemini()

model =  ollama3()
%ai register llama3big model

model =  gpt4()
%ai register gpt4o model

model = opus()
%ai register opus model 

Registered new alias `opus`

# Metrik

Aufgabe: Analysierem welche Silben betont sind und welche unbetont.

## Hälfte des Lebens

Lösung: 
Jede Strophe hat sieben Zeilen, davon drei dreihebige und vier zweihebige.
Erste Strophe: Von den dreihebigen Versen ist in der ersten Strophe ein Paar an den Anfang gesetzt, ein einzelner an den Schluß: Die längeren dreihebigen Verse umrahmen die kürzeren. (Strauss nimmt dafür an, dass der Vers “Es Winter ist, die Blumen, und wo” drei Senkungen hintereinander hat, nämlich “ter ist, die”.)
Zweite Strophe beginnt ebenfalls mit zwei dreihebigen Versen. Der dritte dreihebige Vers ist der vorletzte. 
1. Str: 3-3-2-2-2-2-3
2. Str: 3-3-2-2-2-3-2

In [4]:
prompt = f"""Analyze the scansion of the following poem, i.e. describe which syllables are stressed and which are not. 
Give the answer using the following characters to indicate stressed syllables and not stressed syllables: / for a stressed syllable and - for an unstressed syllable. 
Number the lines of the poem and use these numbers at the beginning of each line of output. At the end add  for each stanza  the numbers of stressed syllables per vers. 
Example: For the two lines “The house is green / the mouse is dead.” the output would look like this:
1: -/-/
2: -/-/
stressed syllables: 2-2

Here is the poem: 
{poem_1.text}
"""
print(prompt)

Analyze the scansion of the following poem, i.e. describe which syllables are stressed and which are not. 
Give the answer using the following characters to indicate stressed syllables and not stressed syllables: / for a stressed syllable and - for an unstressed syllable. 
Number the lines of the poem and use these numbers at the beginning of each line of output. At the end add  for each stanza  the numbers of stressed syllables per vers. 
Example: For the two lines “The house is green / the mouse is dead.” the output would look like this:
1: -/-/
2: -/-/
stressed syllables: 2-2

Here is the poem: 

Mit gelben Birnen hänget
Und voll mit wilden Rosen
Das Land in den See,
Ihr holden Schwäne,
Und trunken von Küssen
Tunkt ihr das Haupt
Ins heilignüchterne Wasser.

Weh mir, wo nehm’ ich, wenn
Es Winter ist, die Blumen, und wo
Den Sonnenschein,
Und Schatten der Erde?
Die Mauern stehn
Sprachlos und kalt, im Winde
Klirren die Fahnen. 




### Llama3:70b

In [5]:
%%ai llama3big
{prompt}

Here is the analysis of the poem's scansion:

1: Mit -gel/ben -Bir/nen -hän/get (/ = stressed, - = unstressed)
2: Und -voll -mit -wil/den -Ro/sen (-/-/-/-/-)
3: Das -Land -in -den -See, (-/-/-/-/-)
4: Ihr -hol/den -Schwä/ne, (-/-/-/-/-)
5: Und -trun/ken -von -Küs/sen (-/-/-/-/-)
6: Tunkt -ihr -das -Haupt (-/-/-/-/-)
7: Ins -heil/i-gnüch/ter/ne -Wasser. (-/-/-/-/-/)
stressed syllables: 4-5-4-4-4-4-5

8: Weh -mir, -wo -nehm’ -ich, -wenn (-/-/-/-/-)
9: Es -Win/ter -ist, -die -Blu/men, (-/-/-/-/-)
10: Und -wo (-/-)
11: Den -Son/nen/schein, (-/-/-/-/-)
12: Und -Schat/ten -der -Er/de? (-/-/-/-/-)
13: Die -Mau/ern -stehn (-/-/-/-/-)
14: Sprach/los -und -kalt, -im -Win/de (-/-/-/-/-)
15: Klir-ren -die -Fah/nen. (-/-/-/-/-/)
stressed syllables: 5-5-2-4-4-4-4-5

### GPT4o

In [6]:
%%ai gpt4o
{prompt}

1: -/-/-/
2: -/-/-/
3: -/-/ 
4: -/-/
5: -/-/-/
6: -/-/
7: -/-/-/-
stressed syllables: 3-3-2-2-3-2-4

8: /-/-/-/
9: -/-/-/-/
10: -/-/
11: -/-/-/
12: -/-/
13: /-/-/-/
14: /-/-/ 
stressed syllables: 4-4-2-3-2-4-3

### Opus

In [7]:
%%ai opus
{prompt}

1: -/--/-
2: -/--/-
3: -/-/
4: -/-/
5: -/--/
6: /-/
7: --/-/

stressed syllables: 2-2-2-2-2-2-2

8: -//-/
9: -/--/-/
10: -/-
11: -/--/
12: -/-/
13: /---//-
14: /--/

stressed syllables: 2-3-2-2-2-3-2

### Gemini 1.5

In [8]:
print(gemini(prompt))


1: - / - / - - -
2: - - / - / - -
3: - / - / - - 
4: / - - - / -
5: - - - / - -
6: - - / - / -
7: - / - - - - - 

8: - / - / - / -
9: - / - - / - , - /
10: - / - - - -
11: - / - - / - - -
12: - / - - -
13: - - - - - / -
14: - - - - / -

stressed syllables: 3-4-3-3-3-3-3
stressed syllables: 4-5-4-4-2-3-3 



### Evaluation and Discussion

The task  actually demands two steps: first detect the scansion, second report it in a summary way. All of the models are far from being perfect in detecting the correct scansion. 
If we just evaluate on the number of stressed syllables (looking at the patterns, not the summaries), this is the error count:<br/>
Llama3: 6<br/>
GPTo: 5<br/>
Opus: 4<br/>
Gemini: 20(!)<br/>

<br/>
Maybe even more interesting: It is immediately obvious, that most of the LLMs are not very good in reporting their own results. With the exception of Opus all report in their summaries repeatedly *more* stressed syllables than they detected. Interestingly they never report less.

## Unsere Toten

In [14]:
prompt = f"""Analyze the scansion of the following poem, i.e. describe which syllables are stressed and which are not. 
Give the answer using the following characters to indicate stressed syllables and not stressed syllables: / for a stressed syllable and - for an unstressed syllable. 
Number the lines of the poem and use these numbers at the beginning of each line of output. At the end add  for each stanza  the numbers of stressed syllables per vers. 
Example: For the two lines “The house is green / the mouse is dead.” the output would look like this:
1: -/-/
2: -/-/
stressed syllables: 2-2

Here is the poem: 
{poem_2.text}
"""
print(prompt)

Analyze the scansion of the following poem, i.e. describe which syllables are stressed and which are not. 
Give the answer using the following characters to indicate stressed syllables and not stressed syllables: / for a stressed syllable and - for an unstressed syllable. 
Number the lines of the poem and use these numbers at the beginning of each line of output. At the end add  for each stanza  the numbers of stressed syllables per vers. 
Example: For the two lines “The house is green / the mouse is dead.” the output would look like this:
1: -/-/
2: -/-/
stressed syllables: 2-2

Here is the poem: 

Von Westen und Osten, von Nord und Süd
Schleppen sich nächtens viele Füße müd,
Füße, vom Wandern wund und zerfetzt,
Langsam bedächtig zur Erde gesetzt,
Müh'n sich im zitternden Mondenschein
Rastlos tief nach Deutschland hinein.
Und wer mit lauschendem Ohr noch wacht
Hört sie in jedweder werdenden Nacht,
Hört dies Schlurfen so müde und schwer,
Hört eine Klage voll wilder Begehr,
Eine Klage s

### Llama3:70b

In [15]:
%%ai llama3big
{prompt}

Here is the analysis of the poem's scansion:

1: Von / Westen - und / Osten -, von / Nord - und / Süd
2: Schleppen - sich / nächtens - viele / Füße / müd,
3: Füße, - vom / Wandern - wund - und / zerfetzt,
4: Langsam - bedächtig - zur / Erde - gesetzt,
5: Müh'n - sich - im / zitternden / Mondenschein
6: Rastlos - tief - nach / Deutschland - hinein.
stressed syllables: 6-7-7-7-8-7

1: Und - wer - mit / lauschendem / Ohr - noch / wacht
2: Hört - sie - in / jedweder - werdenden / Nacht,
3: Hört - dies / Schlurfen - so - müde - und / schwer,
4: Hört - eine / Klage - voll / wilder / Begehr,
5: Eine - Klage - schmerzzerfressen:
6: Nur - nicht - vergessen! - Uns - nicht - vergessen!
stressed syllables: 7-7-8-9-10-9

### GPT4o

In [16]:
%%ai gpt4o
{prompt}

1: -/--/--/-
2: /--//-/
3: /-/-/-/-
4: /--//-/
5: /-/-/-//-/
6: /-/-'
7: /-/-/-
8: /-/-/-
9: /--/--/
10: /--/--/
11: /--/-/
12: /-/-/-

stressed syllables: 2-4-2-4-4-2-2-2-3-3-2-2

### Opus

In [17]:
%%ai opus
{prompt}

Here is the scansion of the poem:

1: -/---/--/
2: /---/--/--/
3: /---/--/
4: /--/---/--/
5: /---/--/--/
6: /--/--/--/
stressed syllables: 3-4-3-4-4-4

7: -/--/---/--/
8: /---/---/--/
9: /--/--/--/
10: /--/--/--/
11: -/--/--/
12: /--/--/--/
stressed syllables: 4-4-4-4-3-4

### Gemini 1.5

In [18]:
gemini(prompt)

'1: -  - / - - / - - / -\n2: - - / - - / - - / -\n3: - - / - - / - / - -\n4: - - / - - / - - / -\n5: - - / - - / - - / - -\n6: - - / - - / - - / - -\nstressed syllables: 4-4-4-4-4-4\n\n7: - / - - / - - / - -\n8: - / - - / - - / - -\n9: - / - - - - / - / -\n10: - / - - / - - / - -\n11: - - / - - / - - \n12: - / - - / - - / - -\nstressed syllables: 4-4-4-4-3-4 \n'